In [105]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn import metrics
from sklearn.svm import LinearSVC
import string
from nltk import pos_tag
from nltk.corpus import stopwords
# nltk.download('stopwords')

from mi_helper import *

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import wordnet
import re

from numpy import array 

In [106]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t")
print(dataset["feedback"].value_counts())
print(dataset.shape)
# data_test = pd.read_csv("test_set.csv", sep = "\t")
# data_test.drop("Unnamed: 0", axis = 1, inplace = True)
# dataset = pd.concat([data_test, dataset])
# dataset.dropna(inplace=True)
dataset.drop(dataset[dataset.rating == 3].index, inplace=True) #droppa recensioni con 3-4 stelle
dataset.drop(dataset[dataset.rating == 4].index, inplace=True) #droppa recensioni con 3-4 stelle
print(dataset.shape)
print(dataset["feedback"].value_counts())

1    2893
0     257
Name: feedback, dtype: int64
(3150, 5)
(2543, 5)
1    2286
0     257
Name: feedback, dtype: int64


In [107]:
custom_stopwords = ['echo',
 'alexa',
 'dot',
 "star",
 'amazon',
 'prime',
 '2nd',
 'generation',
 "fire",
 "stick",
 "firestick",
 '1st',
 '3rd',
 '4th',
 '5th',
 'google',
 'netflix',
 'youtube',
 'philip',
 'tp-link',
 'fourth',
 'roku',
 'i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'nor',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "...",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 'didn',
 'doesn',
 'hadn',
 'hasn',
 'haven',
 'isn',
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 'wasn',
 'weren',
 'won',
 'wouldn']

def negation_handler(sentence):	
    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    return sentence

In [160]:
# levare i numeri
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
    """
    if treebank_tag.startswith('J'):
        return "a"
    elif treebank_tag.startswith('V'):
        return "v"
    elif treebank_tag.startswith('N'):
        return "n"
    elif treebank_tag.startswith('R'):
        return "r"
    else:
        return "n"
        
custom_stopwords = ["echo", "alexa", "dot", "amazon", "prime", "2nd", "generation", "1st", "3rd", "4th", "5th", "google", "netflix", "youtube", "philip", "tp-link"]
pos_list = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]

def tokenize_list_of_text(list_of_text, custom_stopwords = [], pos_filter = False, pos_list = []):
    """Tokenizza tutte le recensioni, pulisce da stopwords, elimina token <= 2 caratteri e lemmatizza. Ritorna la colonna delle recensioni."""

    tokenizer = nltk.tokenize.TweetTokenizer()
    lemmatizer = nltk.WordNetLemmatizer()
    tokenized_reviews = []
    sent_tokenized_reviews = []
    for review in list_of_text: #pulisce le recensioni
        clean_text = ""
        tokens = nltk.tokenize.word_tokenize(review, language='english', preserve_line=False)
        tokens = [w.lower() for w in tokens]
        tokens_pos = pos_tag(tokens)
        lemmatized_tokens = [(lemmatizer.lemmatize(w, get_wordnet_pos(pos)), pos) for w, pos in tokens_pos]
        if pos_filter:
            clean_tokens = [(w, pos) for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>3 and w not in stopwords.words("english") and w not in custom_stopwords and pos in pos_list]
        else:
            clean_tokens = [(w, pos) for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>3 and w not in stopwords.words("english") and w not in custom_stopwords]
        sent_tokenized_reviews.append([w for w, pos in clean_tokens])
        for w, pos in clean_tokens:
            clean_text += " " + w
        tokenized_reviews.append(clean_text)
    
    n_tokens = []
    for sent in sent_tokenized_reviews:
        for w in sent:
            n_tokens.append(w)
    print("total number of tokens extracted are:", len(set(n_tokens)))
    return tokenized_reviews,  sent_tokenized_reviews # ritorna una tupla!

In [161]:
tokenized_reviews, sentences = tokenize_list_of_text(dataset["verified_reviews"], custom_stopwords, pos_filter=False, pos_list = pos_list)

dataset["verified_reviews"] = tokenized_reviews

total number of tokens extracted are: 1942


In [110]:
# CREAZIONE DI UN SAMPLE DATASET BILANCIATO
# prende le prime n recensioni positive di lunghezza maggiore, dove n è il numero di recensioni negative
def create_balanced_dataset(dataset):
    """Bilancia il dataset uniformando il numero di recensioni negative e positive. Prende in input il dataset"""
    reviews_1 = list(dataset[dataset["feedback"] == 1]["verified_reviews"])
    reviews_0 = list(dataset[dataset["feedback"] == 0]["verified_reviews"])
    reviews_1.sort(key=len, reverse = True)
    sample_1 = reviews_1[:len(reviews_0)]
    verified_reviews_sample = []
    feedback_sample = []
    verified_reviews_sample.extend(sample_1)
    verified_reviews_sample.extend(reviews_0)
    feedback_sample.extend([1 for i in range(len(sample_1))])
    feedback_sample.extend([0 for i in range(len(reviews_0))])
    dataset = pd.DataFrame({"verified_reviews":verified_reviews_sample, "feedback": feedback_sample})
    print(dataset["feedback"].value_counts())
    return dataset


dataset = create_balanced_dataset(dataset)

1    257
0    257
Name: feedback, dtype: int64


In [163]:
dataset

,verified_reviews,feedback,len
0,incredible piece technology right center live...,1,234
1,figure hype honestly really want like persona...,1,205
2,recently cord cable television need easy inst...,1,139
3,work great direct tell item show plus best it...,1,115
4,show long love timer cook stave finicky want ...,1,114
...,...,...,...
509,responds even talk unplug feel like,0,8
510,connect phone playlist,0,4
511,negative product terrible sound quality massi...,0,18
512,order,0,1


In [111]:
tokenizer = nltk.tokenize.TweetTokenizer()
cv = CountVectorizer(stop_words="english", ngram_range=(1, 3), tokenizer=tokenizer.tokenize, min_df = 2) #count == frequenza
text_counts = cv.fit_transform(dataset["verified_reviews"]) 
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset["feedback"], test_size = 0.20, random_state=10) #divisione in train e test

In [112]:
text_counts

<514x4894 sparse matrix of type '<class 'numpy.int64'>'
	with 17872 stored elements in Compressed Sparse Row format>

In [113]:
len(cv.vocabulary_)

4894

In [114]:
cv.vocabulary_ # A mapping of terms to feature indices.

{'piece': 2978,
 'technology': 4140,
 'right': 3505,
 'center': 585,
 'living': 2324,
 'room': 3519,
 'island': 2001,
 'kitchen': 2067,
 'counter': 836,
 'mic': 2507,
 'speaker': 3919,
 'direction': 1076,
 'quality': 3245,
 'sound': 3862,
 'quite': 3301,
 'good': 1547,
 'connect': 742,
 'bluetooth': 409,
 'soundbar': 3907,
 'audiophile': 273,
 'equipment': 1245,
 'range': 3329,
 'decent': 943,
 'bass': 329,
 'indoor': 1925,
 'loud': 2382,
 'bother': 445,
 'work': 4801,
 'great': 1589,
 'adjust': 72,
 'volume': 4597,
 'device': 980,
 'imagine': 1896,
 'button': 486,
 'large': 2119,
 'precise': 3130,
 'recommend': 3420,
 'regular': 3436,
 'look': 2354,
 'weight': 4739,
 'material': 2491,
 'order': 2849,
 'come': 673,
 'free': 1462,
 'hue': 1833,
 'bulb': 476,
 'instal': 1947,
 'extra': 1309,
 'buy': 488,
 'floor': 1443,
 'lamp': 2108,
 'turn': 4394,
 'light': 2171,
 'say': 3566,
 'default': 949,
 'second': 3616,
 'dim': 1072,
 'app': 156,
 'create': 851,
 'group': 1636,
 'surprised': 410

In [115]:
#get_feature_names_out([input_features])
#Get output feature names for transformation.

len(cv.get_feature_names_out())

4894

In [116]:
X_train[0,:] # prima recensione

<1x4894 sparse matrix of type '<class 'numpy.int64'>'
	with 79 stored elements in Compressed Sparse Row format>

In [117]:
print(X_train[0,:])

  (0, 3519)	1
  (0, 3919)	1
  (0, 488)	1
  (0, 3566)	1
  (0, 2599)	1
  (0, 2387)	3
  (0, 2994)	1
  (0, 1125)	1
  (0, 4477)	1
  (0, 4582)	1
  (0, 2278)	1
  (0, 3)	1
  (0, 3215)	1
  (0, 4533)	1
  (0, 3013)	1
  (0, 3142)	1
  (0, 285)	1
  (0, 89)	1
  (0, 4006)	1
  (0, 2566)	1
  (0, 2421)	1
  (0, 2164)	1
  (0, 599)	1
  (0, 3643)	1
  (0, 4616)	1
  :	:
  (0, 2168)	1
  (0, 601)	1
  (0, 3645)	1
  (0, 15)	1
  (0, 4010)	1
  (0, 2580)	1
  (0, 4618)	1
  (0, 100)	1
  (0, 3029)	1
  (0, 2674)	1
  (0, 4535)	1
  (0, 4591)	1
  (0, 3531)	1
  (0, 2301)	1
  (0, 3578)	1
  (0, 2396)	1
  (0, 1129)	1
  (0, 1712)	1
  (0, 3154)	1
  (0, 3969)	1
  (0, 4428)	1
  (0, 500)	1
  (0, 824)	1
  (0, 1728)	1
  (0, 2422)	1


In [118]:
cv.inverse_transform(X_train[0,:]) #0 ==> prima recensione

[array(['room', 'speaker', 'buy', 'say', 'music', 'love', 'play', "don't",
        'use', 'voice', 'listen', 'able', 'purchase', 'use voice',
        'play music', 'price', 'awesome', 'alarm', 'start', 'morning',
        'love purchase', 'life', 'changer', 'seriously', 'wakeme',
        'hesitate', 'unbelievable', 'cordless', 'holder', 'awesome life',
        'life changer', 'changer seriously', 'seriously able',
        'able start', 'start morning', 'morning wakeme', 'wakeme alarm',
        'alarm play', 'music use', 'voice room', 'room listen',
        'listen say', 'say love', "love don't", "don't hesitate",
        'hesitate price', 'price speaker', 'speaker unbelievable',
        'unbelievable buy', 'buy cordless', 'cordless holder',
        'holder love', 'purchase love', 'awesome life changer',
        'life changer seriously', 'changer seriously able',
        'seriously able start', 'able start morning',
        'start morning wakeme', 'morning wakeme alarm',
        'wakeme 

In [119]:
for feat,freq in zip(cv.inverse_transform(X_train[0,:])[0],X_train[0,:].data):
    print(feat,freq)

room 1
speaker 1
buy 1
say 1
music 1
love 3
play 1
don't 1
use 1
voice 1
listen 1
able 1
purchase 1
use voice 1
play music 1
price 1
awesome 1
alarm 1
start 1
morning 1
love purchase 1
life 1
changer 1
seriously 1
wakeme 1
hesitate 1
unbelievable 1
cordless 1
holder 1
awesome life 1
life changer 1
changer seriously 1
seriously able 1
able start 1
start morning 1
morning wakeme 1
wakeme alarm 1
alarm play 1
music use 1
voice room 1
room listen 1
listen say 1
say love 1
love don't 1
don't hesitate 1
hesitate price 1
price speaker 1
speaker unbelievable 1
unbelievable buy 1
buy cordless 1
cordless holder 1
holder love 1
purchase love 1
awesome life changer 1
life changer seriously 1
changer seriously able 1
seriously able start 1
able start morning 1
start morning wakeme 1
morning wakeme alarm 1
wakeme alarm play 1
alarm play music 1
play music use 1
music use voice 1
use voice room 1
voice room listen 1
room listen say 1
listen say love 1
say love don't 1
love don't hesitate 1
don't hesi

In [120]:
#For classification we'll set 'chi2'  method as a scoring function. The target number of features is defined by k parameter
select = SelectKBest(chi2, k=4000)  # feature selection
# select = SelectKBest(chi2, k="all")  # feature selection for balanced with pos filter
select.fit(X_train,Y_train)
X_train_sel = select.transform(X_train)
X_test_sel = select.transform(X_test)

In [121]:
#We've selected 3 best features in x data. To identify the selected features we use get_support() function and filter out them from the features name list. The z object contains selected x data
filter = select.get_support() #filtra le parole selezionando quelle contenenti nei k 5000
sum(filter)

4000

In [122]:
X_train_sel

<411x4000 sparse matrix of type '<class 'numpy.int64'>'
	with 11730 stored elements in Compressed Sparse Row format>

In [123]:
print(X_test_sel[0,:])

  (0, 1068)	1
  (0, 931)	1
  (0, 744)	1
  (0, 1159)	1
  (0, 2992)	1
  (0, 3000)	1
  (0, 2159)	1
  (0, 3684)	1
  (0, 3598)	1
  (0, 3601)	1
  (0, 2460)	1


In [124]:
print(cv.inverse_transform(select.inverse_transform(X_train_sel[0,:]))) #applichi l'inverse_transform sui vettori che poi hai utilizzato con il Select

[array(['able', 'able start', 'able start morning', 'alarm', 'alarm play',
       'alarm play music', 'awesome', 'awesome life',
       'awesome life changer', 'buy', 'buy cordless',
       'buy cordless holder', 'changer', 'changer seriously',
       'changer seriously able', 'cordless', 'cordless holder',
       'cordless holder love', "don't", "don't hesitate",
       "don't hesitate price", 'hesitate', 'hesitate price',
       'hesitate price speaker', 'holder', 'holder love',
       'holder love purchase', 'life', 'life changer',
       'life changer seriously', 'listen', 'listen say',
       'listen say love', 'love', "love don't", "love don't hesitate",
       'love purchase', 'love purchase love', 'morning', 'morning wakeme',
       'morning wakeme alarm', 'music', 'music use', 'music use voice',
       'play', 'play music', 'play music use', 'price', 'price speaker',
       'price speaker unbelievable', 'purchase', 'purchase love', 'room',
       'room listen', 'room listen sa

In [125]:
tfidf = TfidfTransformer()  # weighting
tfidf.fit(X_train_sel)
X_train_vec = tfidf.transform(X_train_sel)
X_test_vec =tfidf.transform(X_test_sel)

In [126]:
print(X_train_vec[0,:])

  (0, 3754)	0.1233935465782052
  (0, 3753)	0.1233935465782052
  (0, 3752)	0.1233935465782052
  (0, 3730)	0.1233935465782052
  (0, 3729)	0.1233935465782052
  (0, 3723)	0.07685661035454135
  (0, 3679)	0.1233935465782052
  (0, 3677)	0.10895181194789495
  (0, 3628)	0.05033434639794235
  (0, 3585)	0.1233935465782052
  (0, 3584)	0.1233935465782052
  (0, 3583)	0.1233935465782052
  (0, 3193)	0.1233935465782052
  (0, 3192)	0.1233935465782052
  (0, 3156)	0.1233935465782052
  (0, 3155)	0.1233935465782052
  (0, 3108)	0.05963274663075159
  (0, 2856)	0.1233935465782052
  (0, 2855)	0.1233935465782052
  (0, 2854)	0.1233935465782052
  (0, 2797)	0.1233935465782052
  (0, 2796)	0.1233935465782052
  (0, 2788)	0.07104961133722319
  (0, 2758)	0.1233935465782052
  (0, 2757)	0.1233935465782052
  :	:
  (0, 1156)	0.11739968515199256
  (0, 1147)	0.1233935465782052
  (0, 1146)	0.1233935465782052
  (0, 1145)	0.1233935465782052
  (0, 748)	0.1233935465782052
  (0, 747)	0.1233935465782052
  (0, 744)	0.0754191425708698

In [127]:
cv.inverse_transform(select.inverse_transform(X_train_vec[0,:]))

[array(['able', 'able start', 'able start morning', 'alarm', 'alarm play',
        'alarm play music', 'awesome', 'awesome life',
        'awesome life changer', 'buy', 'buy cordless',
        'buy cordless holder', 'changer', 'changer seriously',
        'changer seriously able', 'cordless', 'cordless holder',
        'cordless holder love', "don't", "don't hesitate",
        "don't hesitate price", 'hesitate', 'hesitate price',
        'hesitate price speaker', 'holder', 'holder love',
        'holder love purchase', 'life', 'life changer',
        'life changer seriously', 'listen', 'listen say',
        'listen say love', 'love', "love don't", "love don't hesitate",
        'love purchase', 'love purchase love', 'morning', 'morning wakeme',
        'morning wakeme alarm', 'music', 'music use', 'music use voice',
        'play', 'play music', 'play music use', 'price', 'price speaker',
        'price speaker unbelievable', 'purchase', 'purchase love', 'room',
        'room listen', 

In [128]:
for feat,weight,freq in zip(cv.inverse_transform(select.inverse_transform(X_train_vec[1,:]))[0],X_train_vec[1,:].data,X_train_sel[1,:].data):
    print(feat,round(weight,4),freq)

anymore 0.1327 1
anymore play 0.1453 1
anymore play need 0.1453 1
cd's 0.1075 1
cd's anymore 0.1453 1
cd's anymore play 0.1453 1
easy 0.1453 1
easy like 0.1453 1
easy like personal 0.0865 1
gadget 0.1453 1
gadget make 0.0647 1
gadget make life 0.1453 1
hardly need 0.1453 1
hardly need cd's 0.1245 1
home 0.1453 1
home hardly 0.1453 1
home hardly need 0.0836 1
life 0.1453 1
life easy like 0.1453 1
like 0.1453 1
like personal 0.1453 1
like personal secretary 0.1283 1
little 0.0773 1
little gadget 0.1453 1
little gadget make 0.1283 1
living 0.1453 1
living home 0.1327 1
living home hardly 0.1152 1
love 0.1453 2
love love 0.1453 1
love love new 0.1113 1
love new 0.1453 1
love new little 0.1382 1
make 0.0836 1
make life 0.1453 1
need cd's 0.1453 1
need cd's anymore 0.0618 1
need soon 0.1453 1
need soon room 0.1113 1
new 0.1453 1
new little 0.1453 1
new little gadget 0.0783 1
personal 0.1453 1
personal secretary 0.1453 1
personal secretary living 0.1453 1
play 0.1453 1
play need soon 0.1382 1

In [129]:
# create a dataframe with words, tf_idf score and freq
total_scores = []

for i in range(X_train_vec.shape[0]):
    for feat,weight,freq in zip(cv.inverse_transform(select.inverse_transform(X_train_vec[i,:]))[0],X_train_vec[i,:].data,X_train_sel[i,:].data):
        total_scores.append((feat,round(weight,4),freq))

words = []
tf_idf_score = []
freq = []
for i in range(len(total_scores)):
    words.append(total_scores[i][0])
    tf_idf_score.append(total_scores[i][1])
    freq.append(total_scores[i][2])

scores_df = pd.DataFrame()
scores_df["words"] = words
scores_df["tf_idf_score"] = tf_idf_score
scores_df["freq"] = freq

In [130]:
scores_df.sort_values(by = "tf_idf_score", ascending = False).head(50) # sort and show the df

,words,tf_idf_score,freq
4342,volume low,1.0000,1
9518,order,1.0000,1
10127,feature,1.0000,1
5321,volume low,1.0000,1
4105,dumb,1.0000,1
6178,feature,1.0000,1
5404,malfunction,1.0000,1
9080,disconnect,1.0000,1
806,suck,1.0000,1
4921,turn,1.0000,1


In [131]:
svm = LinearSVC()  # linear svm with default parameters
svm_clf = svm.fit(X_train_vec,Y_train)
predictions = svm_clf.predict(X_test_vec)

In [132]:
len(predictions), sum(predictions)

(103, 50)

In [133]:
predictions

array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1], dtype=int64)

In [134]:
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.91      0.94        56
           1       0.90      0.96      0.93        47

    accuracy                           0.93       103
   macro avg       0.93      0.93      0.93       103
weighted avg       0.93      0.93      0.93       103



In [135]:
feature_names = cv.get_feature_names_out()
feature_names

array(['ability', 'ability play', 'ability play music', ..., 'z-wave',
       'zero', 'zigbee'], dtype=object)

In [136]:
print(select.scores_)

[1.82953092e+00 9.59488273e-04 9.59488273e-04 ... 2.08955224e+00
 2.08955224e+00 4.17910448e+00]


In [137]:
feature_names = cv.get_feature_names_out()
feats_w_score = list()
for index,(selected,score) in enumerate(zip(filter,select.scores_)):
    feats_w_score.append((score,selected,feature_names[index]))
feats_w_score = sorted(feats_w_score, reverse = True)
len(feats_w_score)

4894

In [138]:
feats_w_score[:10] # this contains k score, wether it is a selcted feature and the word

[(49.21201932395962, True, 'music'),
 (46.02133211493552, True, 'great'),
 (39.94237307670143, True, 'play'),
 (25.13896178448417, True, 'use'),
 (24.885714285714286, True, 'enjoy'),
 (24.56529964160958, True, 'alarm'),
 (23.313241484883278, True, 'weather'),
 (23.300239872068225, True, 'easy'),
 (23.028639963448065, True, 'learn'),
 (22.971428571428568, True, 'house')]

In [139]:
type(svm)

sklearn.svm._classes.LinearSVC

In [140]:
svm.coef_

array([[ 0.39040719, -0.22250773,  0.1322514 , ..., -0.11216478,
        -0.43124975, -0.13291644]])

In [141]:
feats_w_classifier_weight = list()
for index,weight in enumerate(select.inverse_transform(svm.coef_)[0]):
    if weight!=0:
        feats_w_classifier_weight.append((weight,feature_names[index]))
feats_w_classifier_weight = sorted(feats_w_classifier_weight)
len(feats_w_classifier_weight)

3989

In [142]:
feats_w_classifier_weight[-100:] #features positive

[(0.3953525953588534, 'wife'),
 (0.3964623376350392, 'way'),
 (0.3973394332314925, 'plan'),
 (0.39765725464473045, 'place'),
 (0.3989788212911496, 'trial'),
 (0.3992033463060369, 'privacy'),
 (0.4039701032127933, 'smart plug'),
 (0.4053773828222034, 'listen music'),
 (0.4082929047420726, 'floor'),
 (0.41503782780072274, 'mom'),
 (0.4151733920560538, 'morning'),
 (0.41583101036633807, 'quite'),
 (0.4169659824755729, 'spell'),
 (0.4206009153577103, 'hook'),
 (0.4221946899616709, 'range'),
 (0.4240894777294064, 'brand new'),
 (0.42938853436752006, 'search'),
 (0.42991929771524345, 'station'),
 (0.43335724142506493, 'love thing'),
 (0.43421849014461544, 'mother'),
 (0.4343934753270535, 'breeze'),
 (0.4351132478817246, 'happy'),
 (0.4383150658919929, 'highly'),
 (0.4383150658919929, 'highly recommend'),
 (0.43974907925193796, 'nice'),
 (0.44401934828757655, 'drive'),
 (0.4499767727687118, 'time way'),
 (0.45394147331499807, 'bedroom'),
 (0.45684171536557217, 'voice'),
 (0.4633060070075735, 

In [143]:
feats_w_classifier_weight[:100] #features negative

[(-1.2468794766358495, 'return'),
 (-0.8960041381375227, 'money'),
 (-0.8362331575079458, 'send'),
 (-0.749434100225033, 'disappointed'),
 (-0.7288295928814311, 'poor'),
 (-0.7193473592311902, 'realize'),
 (-0.6956056626956023, 'unable'),
 (-0.691175765292373, 'useless'),
 (-0.6743200357381263, 'terrible'),
 (-0.6181012553862556, 'sonos'),
 (-0.6093808285393707, 'try'),
 (-0.6076730260873017, "can't"),
 (-0.6073168962268843, 'hard'),
 (-0.5996948455642718, 'ignore'),
 (-0.5969590602723847, 'difference'),
 (-0.5788527227644702, 'half'),
 (-0.5540660737768861, 'want use'),
 (-0.5532825554411912, 'sound like'),
 (-0.5412125234593791, 'support'),
 (-0.5340537960280719, 'firmware'),
 (-0.5336230109510895, 'sound terrible'),
 (-0.5327452072796279, 'stop work'),
 (-0.527501694217412, 'allow'),
 (-0.5274017959104115, 'month'),
 (-0.5039546221294838, 'awful'),
 (-0.4894222167835669, 'look like'),
 (-0.47543402207127344, 'figure use'),
 (-0.4636848999529463, 'bad'),
 (-0.4382882848031669, 'fan')

In [144]:
df_scores = pd.DataFrame()
scores = []
words = []
for entry in feats_w_classifier_weight:
    scores.append(entry[0])
    words.append(entry[1])

df_scores["scores"] = scores
df_scores["words"] = words

df_scores.sort_values(by = "scores", ascending = False).head(15) # use tail for seeing the positive words

,scores,words
3988,2.019992,love
3987,1.395097,great
3986,1.167876,little
3985,1.138545,enjoy
3984,0.992324,command
3983,0.961269,music
3982,0.918498,learn
3981,0.912699,camera
3980,0.907330,play
3979,0.857434,attention


In [145]:
stringa = "the product is disappointing, audio sounds bad"
clean = tokenize_list_of_text([stringa])[0]
clean

total number of tokens extracted are: 6


['the product disappointing audio sound bad']

In [146]:
vector = cv.transform(clean)
vector = select.transform(vector)
vector = tfidf.transform(vector)
predicted = svm.predict(vector)
print(predicted)

[0]


In [147]:
MNB = MultinomialNB()  # MNB with default parameters
MNB_clsf = MNB.fit(X_train_vec,Y_train)
predictions = MNB_clsf.predict(X_test_vec)
print(metrics.classification_report(predictions, Y_test))

              precision    recall  f1-score   support

           0       0.64      0.97      0.77        37
           1       0.98      0.70      0.81        66

    accuracy                           0.80       103
   macro avg       0.81      0.83      0.79       103
weighted avg       0.86      0.80      0.80       103



In [148]:
len(Y_test)

103